In [4]:
import mysql.connector as mysql
from mysql.connector import errorcode

ds_host = "localhost"
db_username = "root"
db_password = "qiwoqqwu"

try:
    conn = mysql.connect(
        host=ds_host,
        port=3306,
        user=db_username,
        password=db_password,
        database = "week10_store_procedure"
    )
    cursor = conn.cursor()


except mysql.Error as err:
    print(" Lỗi:", err)


In [2]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS order_logs (
        log_id INT AUTO_INCREMENT PRIMARY KEY,
        order_id INT,
        deleted_at DATETIME,
        message VARCHAR(255)
    )
""")
conn.commit()


In [3]:
cursor.execute("DROP TRIGGER IF EXISTS after_order_delete")

cursor.execute("""
DELIMITER $$
CREATE TRIGGER after_order_delete
AFTER DELETE ON orders
FOR EACH ROW
BEGIN
    INSERT INTO order_logs(order_id, deleted_at, message)
    VALUES (
        OLD.id,
        NOW(),
        CONCAT('Đơn hàng ', OLD.id, ' đã bị xoá lúc ', NOW())
    );
END$$
DELIMITER ;
""")
conn.commit()


In [6]:
# Describe cấu trúc bảng order_items
cursor.execute("DESCRIBE order_items")
columns = cursor.fetchall()

print("Cấu trúc bảng 'order_items':")
for col in columns:
    field, dtype, null, key, default, extra = col
    print(f"- {field:15} | {dtype:15} | Null: {null:3} | Key: {key:3} | Default: {default} | Extra: {extra}")


Cấu trúc bảng 'order_items':
- id              | int             | Null: NO  | Key: PRI | Default: None | Extra: 
- order_id        | int             | Null: YES | Key: MUL | Default: None | Extra: 
- product_id      | int             | Null: YES | Key: MUL | Default: None | Extra: 
- quantity        | int             | Null: YES | Key:     | Default: None | Extra: 


In [11]:
def show_order_logs():
    cursor.execute("SELECT * FROM order_logs ORDER BY deleted_at DESC")
    logs = cursor.fetchall()
    print("[Order Logs] Danh sách các đơn hàng đã bị xoá:")
    for log in logs:
        log_id, order_id, deleted_at, message = log
        print(f"- [log_id {log_id}] {message}")


In [16]:
def delete_order(order_id):
    # 1. Lấy dữ liệu đơn hàng và các mặt hàng trước khi xoá
    cursor.execute("SELECT * FROM orders WHERE id = %s", (order_id,))
    order_data = cursor.fetchone()

    cursor.execute("SELECT * FROM order_items WHERE order_id = %s", (order_id,))
    order_items_data = cursor.fetchall()

    if not order_data:
        print(f"Không tìm thấy đơn hàng với id = {order_id}")
        return

    # 2. Xoá: order_items trước rồi đến orders
    cursor.execute("DELETE FROM order_items WHERE order_id = %s", (order_id,))
    cursor.execute("DELETE FROM orders WHERE id = %s", (order_id,))
    conn.commit()
    print(f"Đã xoá đơn hàng {order_id}")

    # 3. Ghi log đã thực hiện trigger (sẽ tự động thực hiện)
    show_order_logs()

In [12]:
# Gọi hàm delete_order với ID đơn hàng cụ thể
order_id_to_delete = 1  # ID của đơn hàng bạn muốn xoá
delete_order(order_id_to_delete)


Không tìm thấy đơn hàng với id = 1


In [27]:
def create_trigger_update_total():
    cursor.execute("DROP TRIGGER IF EXISTS update_order_total")

    trigger_sql = """
    CREATE TRIGGER update_order_total
    AFTER INSERT ON order_items
    FOR EACH ROW
    BEGIN
        UPDATE orders
        SET total = (
            SELECT SUM(oi.quantity * p.price)
            FROM order_items AS oi
            JOIN products AS p ON oi.product_id = p.id
            WHERE oi.order_id = NEW.order_id
        )
        WHERE id = NEW.order_id;
    END;
    """
    
    cursor.execute(trigger_sql)  # Chạy trigger SQL
    conn.commit()


In [32]:
def add_order_item(order_id, product_id, quantity):
    cursor.execute("INSERT INTO order_items (order_id, product_id, quantity) VALUES (%s, %s, %s)", 
                   (order_id, product_id, quantity))
    conn.commit()
    print("Order item added successfully.")

In [34]:
cursor.execute("ALTER TABLE order_items MODIFY id INT NOT NULL AUTO_INCREMENT;")
conn.commit()
print("Table 'order_items' modified successfully.")


Table 'order_items' modified successfully.


In [33]:
create_trigger_update_total()
add_order_item(2, 1, 2)

Order item added successfully.


In [35]:
cursor.execute("SELECT total FROM orders WHERE id = 2;")
result = cursor.fetchone()
print("Updated total for order_id 2:", result[0])


Updated total for order_id 2: 4800


In [6]:
# Tạo trigger ngăn không cho cập nhật giá sản phẩm
def create_trigger_prevent_price_update():
    cursor.execute("DROP TRIGGER IF EXISTS prevent_price_update;")  # Xóa trigger nếu đã tồn tại
    cursor.execute("""
        CREATE TRIGGER prevent_price_update
        BEFORE UPDATE ON products
        FOR EACH ROW
        BEGIN
            IF NEW.price > 1000 THEN
                SIGNAL SQLSTATE '45000' SET MESSAGE_TEXT = 'Price cannot be more than 1000';
            END IF;
        END;
    """)
    conn.commit()
    print("Trigger to prevent price update created.")

# Hàm cập nhật giá sản phẩm
def update_product_price(product_id, new_price):
    cursor.execute("UPDATE products SET price = %s WHERE id = %s", (new_price, product_id))
    conn.commit()
    print("Product price updated successfully.")

# Đảm bảo bạn đã kết nối và tạo `cursor` ở trên trước khi gọi hàm này
create_trigger_prevent_price_update()



Trigger to prevent price update created.


In [8]:
# Thử cập nhật giá sản phẩm
try:
    update_product_price(1, 900)
except mysql.connector.Error as e:
    print("Trigger blocked the update:", e)


Product price updated successfully.


In [12]:
def create_trigger_log_customer_delete():
    cursor.execute("DROP TRIGGER IF EXISTS log_customer_delete;")
    cursor.execute("""
        CREATE TRIGGER log_customer_delete
        AFTER DELETE ON customers
        FOR EACH ROW
        BEGIN
            INSERT INTO customers_deleted (id, name, city)
            VALUES (OLD.id, OLD.name, OLD.city);
        END;
    """)
    conn.commit()
    print("Trigger log_customer_delete created.")



In [20]:
# Tạo bảng customers_deleted nếu chưa có
cursor.execute("""
    CREATE TABLE IF NOT EXISTS customers_deleted (
        id INT NOT NULL,
        name VARCHAR(255) NOT NULL,
        city VARCHAR(255),
        PRIMARY KEY (id)
    );
""")
conn.commit()

# Sau đó bạn có thể chạy trigger và các thao tác xóa khách hàng
create_trigger_log_customer_delete()
delete_customer(1)


Trigger log_customer_delete created.
🧹 Deleted order items for customer 1
🧹 Deleted order assignments for customer 1
🧹 Deleted all orders of customer 1
🧹 Customer with ID 1 has been deleted.


In [21]:
def delete_orders_of_customer(customer_id):
    # Xóa các bản ghi trong bảng order_items liên quan đến đơn hàng của khách
    cursor.execute("DELETE FROM order_items WHERE order_id IN (SELECT id FROM orders WHERE customer = %s)", (customer_id,))
    conn.commit()
    print(f"🧹 Deleted order items for customer {customer_id}")

    # Xóa các bản ghi trong bảng order_assignments liên quan đến đơn hàng của khách
    cursor.execute("DELETE FROM order_assignments WHERE order_id IN (SELECT id FROM orders WHERE customer = %s)", (customer_id,))
    conn.commit()
    print(f"🧹 Deleted order assignments for customer {customer_id}")

    # Xóa các đơn hàng của khách
    cursor.execute("DELETE FROM orders WHERE customer = %s", (customer_id,))
    conn.commit()
    print(f"🧹 Deleted all orders of customer {customer_id}")

def delete_customer(customer_id):
    # Xóa các đơn hàng và liên kết trước
    delete_orders_of_customer(customer_id)
    
    # Xóa khách hàng và ghi log
    cursor.execute("DELETE FROM customers WHERE id = %s", (customer_id,))
    conn.commit()
    print(f"🧹 Customer with ID {customer_id} has been deleted.")

# Test lại (Xóa khách hàng có ID 1)
delete_customer(1)


🧹 Deleted order items for customer 1
🧹 Deleted order assignments for customer 1
🧹 Deleted all orders of customer 1
🧹 Customer with ID 1 has been deleted.


In [22]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS order_alerts (
        id INT AUTO_INCREMENT PRIMARY KEY,
        order_id INT NOT NULL,
        customer_id INT NOT NULL,
        log_message TEXT,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );
""")
conn.commit()


In [40]:
def create_trigger_alert_order_above_1000():
    cursor.execute("DROP TRIGGER IF EXISTS alert_order_above_1000;")
    cursor.execute("""
        CREATE TRIGGER alert_order_above_1000
        AFTER INSERT ON orders
        FOR EACH ROW
        BEGIN
            IF NEW.total > 1000 AND 
               EXISTS (SELECT 1 FROM customers WHERE id = NEW.customer AND city = 'Hanoi') THEN
                INSERT INTO order_alerts (order_id, customer_id, log_message)
                VALUES (
                    NEW.id,
                    NEW.customer,  -- Sửa lại từ customer_id thành customer
                    'Alert: Order total above 1000 from Hanoi'
                );
            END IF;
        END;
    """)
    conn.commit()
    print("Trigger alert_order_above_1000 created.")


In [39]:
# Xóa ràng buộc khóa ngoại trong bảng 'order_assignments' trước khi thay đổi cột 'id' trong bảng 'orders'
try:
    cursor.execute("ALTER TABLE order_assignments DROP FOREIGN KEY order_assignments_ibfk_1;")
    conn.commit()
    print("Đã xóa ràng buộc khóa ngoại.")
except mysql.connector.Error as err:
    print(f"Lỗi khi xóa ràng buộc khóa ngoại: {err}")

# Thực hiện thay đổi cột 'id' trong bảng 'orders'
try:
    cursor.execute("ALTER TABLE orders MODIFY COLUMN id INT AUTO_INCREMENT;")
    conn.commit()
    print("Cột 'id' trong bảng 'orders' đã được sửa thành công.")
except mysql.connector.Error as err:
    print(f"Lỗi khi sửa cột 'id': {err}")

# Thêm lại ràng buộc khóa ngoại sau khi thay đổi
try:
    cursor.execute("ALTER TABLE order_assignments ADD CONSTRAINT order_assignments_ibfk_1 FOREIGN KEY (order_id) REFERENCES orders(id);")
    conn.commit()
    print("Ràng buộc khóa ngoại đã được thêm lại.")
except mysql.connector.Error as err:
    print(f"Lỗi khi thêm lại ràng buộc khóa ngoại: {err}")


Đã xóa ràng buộc khóa ngoại.
Cột 'id' trong bảng 'orders' đã được sửa thành công.
Ràng buộc khóa ngoại đã được thêm lại.


In [41]:
def add_order(customer_id, order_date, total):
    cursor.execute(
        "INSERT INTO orders (customer, order_date, total) VALUES (%s, %s, %s)",  # Không cần đưa id vào
        (customer_id, order_date, total)
    )
    order_id = cursor.lastrowid  # Lấy id của đơn hàng vừa chèn
    conn.commit()

    cursor.execute("SELECT log_message FROM order_alerts WHERE order_id = %s", (order_id,))
    alerts = cursor.fetchall()
    for alert in alerts:
        print(alert[0])


In [ ]:
# Thêm đơn hàng từ khách ở Hà Nội để test
add_order(3, "2024-01-10", 1200)


In [31]:
cursor.execute("DESCRIBE orders;")
columns = cursor.fetchall()
for column in columns:
    print(column)


('id', 'int', 'NO', 'PRI', None, '')
('customer', 'int', 'YES', 'MUL', None, '')
('order_date', 'date', 'YES', '', None, '')
('total', 'int', 'YES', '', None, '')
